In [3]:
#Importing necessary libraries

from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")


Data Loading and Initial Exploration

In [4]:
#Initialize Spark Session
spark = SparkSession.builder.appName("FlightDelayPrediction").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/24 22:55:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
#Load the dataset
df = spark.read.csv("../Dataset/Combined_Flights_2022.csv",header=True,inferSchema=True)

In [6]:
#Partitioning the dataset
df = df.repartition(5)

In [19]:
#Display schema and first 10 rows
df.printSchema()
df.show(20)

root
 |-- FlightDate: date (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- ArrDelayMinutes: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |

24/10/24 23:13:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:13:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:13:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:13:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:13:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:13:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:13:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:13:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:13:37 WARN RowBasedKeyValueBatch: Calling spill() on

+----------+--------------------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+--------------------+---------+-------------+--------------+-------+--------+--------------------+----------+-------+---------+--------+------+----------+--------+--------+------------------+----------+-------------+------------------+
|FlightDate|             Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|DepDelayMinutes|DepDel

In [8]:
#Count the total number of rows
row_count = df.count()
print("Total rows:", row_count)

Total rows: 4078318


Data Cleaning and Transformation

In [9]:
from pyspark.sql.functions import col, isnan, when, count

In [10]:
df =df.dropna()

In [11]:
df.count()

3944916

In [12]:
# Remove duplicates
df = df.dropDuplicates()

In [13]:
# Filter invalid rows: For example, remove rows where delays are negative
df = df.filter((df["DepDelayMinutes"]>= 0) & (df["ArrDelayMinutes"]>= 0))

In [14]:
df.show()

24/10/24 22:56:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:56:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:56:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:56:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:56:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:56:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:56:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:56:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:56:36 WARN RowBasedKeyValueBatch: Calling spill() on

+----------+--------------------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+--------------------+---------+-------------+--------------+-------+--------+--------------------+----------+-------+---------+--------+------+----------+--------+--------+------------------+----------+-------------+------------------+
|FlightDate|             Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|DepDelayMinutes|DepDel

Data Analysis Using Spark SQL

In [15]:
# Register the cleaned DataFrame as a SQL temporary view

df.createOrReplaceTempView("flight_data")

In [16]:
# Aggregation: Calculate summary statistics for delays
spark.sql("""
    SELECT 
        AVG(DepDelayMinutes) AS avg_dep_delay,
        STDDEV(DepDelayMinutes) AS stddev_dep_delay,
        AVG(ArrDelayMinutes) AS avg_arr_delay,
        STDDEV(ArrDelayMinutes) AS stddev_arr_delay
    FROM flight_data
""").show()

24/10/24 22:57:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:03 WARN RowBasedKeyValueBatch: Calling spill() on

+------------------+-----------------+------------------+-----------------+
|     avg_dep_delay| stddev_dep_delay|     avg_arr_delay| stddev_arr_delay|
+------------------+-----------------+------------------+-----------------+
|15.940134086505264|52.16239928825902|15.783071426615928|51.98423581313933|
+------------------+-----------------+------------------+-----------------+



In [17]:
# Grouping: Average departure delay by airline

spark.sql("""
    SELECT 
        Airline, 
        AVG(DepDelayMinutes) AS avg_dep_delay
    FROM flight_data
    GROUP BY Airline
    ORDER BY avg_dep_delay DESC
""").show()

24/10/24 22:57:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:57:41 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+------------------+
|             Airline|     avg_dep_delay|
+--------------------+------------------+
|     JetBlue Airways|28.913369286392374|
|Frontier Airlines...|  23.4853028822085|
|       Allegiant Air|22.665417589280644|
|GoJet Airlines, L...|20.106726957383547|
|  Mesa Airlines Inc.|18.805355872144528|
|    Spirit Air Lines| 18.36243988457839|
|American Airlines...|17.632990873577082|
|         Comair Inc.|   16.827018096592|
|United Air Lines ...|16.338848566321307|
|Commutair Aka Cha...|16.277818096687348|
|Southwest Airline...| 15.92439971582852|
|SkyWest Airlines ...|14.935353732532983|
|   Republic Airlines| 14.70693922794632|
|Delta Air Lines Inc.|13.741698787721436|
|   Endeavor Air Inc.|13.177635965330031|
|Air Wisconsin Air...|13.055913756641049|
|Capital Cargo Int...|11.978940164254986|
|           Envoy Air| 10.12833114897335|
|Alaska Airlines Inc.|10.069359506932761|
|         Horizon Air|  8.86154960173787|
+--------------------+------------

In [18]:

# Time-based analysis: Delay trends by month
spark.sql("""
    SELECT 
        Year, Month, 
        AVG(DepDelayMinutes) AS avg_dep_delay,
        AVG(ArrDelayMinutes) AS avg_arr_delay
    FROM flight_data
    GROUP BY Year, Month
    ORDER BY Year, Month
""").show()

24/10/24 23:00:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:00:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:00:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:00:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:00:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:00:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:00:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:00:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:00:36 WARN RowBasedKeyValueBatch: Calling spill() on

+----+-----+------------------+------------------+
|Year|Month|     avg_dep_delay|     avg_arr_delay|
+----+-----+------------------+------------------+
|2022|    1|14.554631481854035|14.354921483258492|
|2022|    2| 14.38924364119108| 14.22176289364896|
|2022|    3| 15.23190499919821|15.282290116612899|
|2022|    4|15.979667586794719|15.949220248947393|
|2022|    5| 15.19160603757194|15.079478974372895|
|2022|    6|18.224888112801544|17.849852879505566|
|2022|    7|17.589301722785844|17.326682130740213|
+----+-----+------------------+------------------+



Insights or patterns observed

In [20]:
# Delay patterns by month
spark.sql("""
    SELECT 
        Year, Month, 
        AVG(DepDelayMinutes) AS avg_dep_delay,
        AVG(ArrDelayMinutes) AS avg_arr_delay
    FROM flight_data
    GROUP BY Year, Month
    ORDER BY Year, Month
""").show()

24/10/24 23:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:35:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 23:35:35 WARN RowBasedKeyValueBatch: Calling spill() on

+----+-----+------------------+------------------+
|Year|Month|     avg_dep_delay|     avg_arr_delay|
+----+-----+------------------+------------------+
|2022|    1|14.554631481854035|14.354921483258492|
|2022|    2| 14.38924364119108| 14.22176289364896|
|2022|    3| 15.23190499919821|15.282290116612899|
|2022|    4|15.979667586794719|15.949220248947393|
|2022|    5| 15.19160603757194|15.079478974372895|
|2022|    6|18.224888112801544|17.849852879505566|
|2022|    7|17.589301722785844|17.326682130740213|
+----+-----+------------------+------------------+

